In [ ]:
# 2023-10-24

import duckdb as db
import pandas as pd

In [ ]:
con = db.connect(
    "/Users/jonathan/mres_thesis/wine_analysis_hplc_uv/src/wine_analysis_hplc_uv/notebooks/testdb.db"
)

In [ ]:
con.sql("""--sql
show tables
""")

In [ ]:
con.sql("""--sql
CREATE OR REPLACE VIEW raw AS
SELECT 
            st.detection,
            st.samplecode,
            ct.wine,
            ct.color,
            ct.varietal,
            chm.id,
            cs.*
FROM
c_sample_tracker st
    INNER JOIN c_cellar_tracker ct ON st.ct_wine_name = ct.wine
    LEFT JOIN c_chemstation_metadata chm ON
    (
    chm.join_samplecode=st.samplecode
    )
    LEFT JOIN chromatogram_spectra cs ON (chm.id=cs.id)
    WHERE
    st.detection='raw'
""")

In [ ]:
data = con.sql("""--sql
SELECT * FROM raw
""").df()

In [ ]:
data.columns.get_loc("id")

In [ ]:
# drop wavelengths > 400 as not all samples recorded between 400 and 600nm
idx = data.columns.get_loc("400")


data = data.loc[:, data.columns[: idx + 1]]
data.head()

In [ ]:
# identify the na's. There are 12 rows with NA in the set.
isnamask = data.isna().any(axis=1)
isnamask
data.loc[lambda df: df.isna().any(axis=1)]

# these are random rows, possibly created as these runs are shorter than the maximum? Anyway, drop them

In [ ]:
data = data.dropna()
data

In [ ]:
# write to parquet file before going any further.

path = "/Users/jonathan/mres_thesis/wine_analysis_hplc_uv/src/wine_analysis_hplc_uv/notebooks/tidy_3d_dset_raw.parquet"

data.to_parquet(path)

In [ ]:
pd.read_parquet(path)